# Introduction to Cross-Validation - Lab

## Introduction

In this lab, you'll be able to practice your cross-validation skills!


## Objectives

You will be able to:

- Compare the results with normal holdout validation
- Apply 5-fold cross validation for regression

## Let's get started

This time, let's only include the variables that were previously selected using recursive feature elimination. We included the code to preprocess below.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.datasets import load_boston

boston = load_boston()

boston_features = pd.DataFrame(boston.data, columns = boston.feature_names)
b = boston_features["B"]
logdis = np.log(boston_features["DIS"])
loglstat = np.log(boston_features["LSTAT"])

# minmax scaling
boston_features["B"] = (b-min(b))/(max(b)-min(b))
boston_features["DIS"] = (logdis-min(logdis))/(max(logdis)-min(logdis))

#standardization
boston_features["LSTAT"] = (loglstat-np.mean(loglstat))/np.sqrt(np.var(loglstat))

In [12]:
X = boston_features
y = pd.DataFrame(boston.target,columns = ['target'])

## Train test split

Perform a train-test-split with a test set of 0.20.

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [15]:
print(len(X_train), len(X_test), len(y_train), len(y_test))

404 102 404 102


Fit the model and apply the model to the make test set predictions

In [16]:
from sklearn.linear_model import LinearRegression
linreg=LinearRegression()
linreg.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

Calculate the residuals and the mean squared error

In [17]:
y_hat_train = linreg.predict(X_train)
y_hat_test = linreg.predict(X_test)
train_residuals = y_hat_train - y_train
test_residuals = y_hat_test - y_test

from sklearn.metrics import mean_squared_error
train_mse = mean_squared_error(y_train, y_hat_train)
test_mse = mean_squared_error(y_test, y_hat_test)

In [32]:
print(test_mse)

19.022337999169526


## Cross-Validation: let's build it from scratch!

### Create a cross-validation function

Write a function k-folds that splits a dataset into k evenly sized pieces.
If the full dataset is not divisible by k, make the first few folds one larger then later ones.

We want the folds to be a list of subsets of data!

In [18]:
def kfolds(data, k):
    # Force data as pandas dataframe
    data = pd.DataFrame(data)
    number_obs = len(data)
    fold_size = number_obs//k
    leftovers = number_obs%k
    folds = []
    start_obs = 0
    for fold_n in range(1, k+1):
        if fold_n <= leftovers:
            fold = data.iloc[start_obs: start_obs+fold_size+1]
            folds.append(fold)
            start_obs +=  fold_size + 1
        else:
            fold = data.iloc[start_obs: start_obs+fold_size]
            folds.append(fold)
            start_obs +=  fold_size
    return folds

### Apply it to the Boston Housing Data

In [19]:
# Make sure to concatenate the data again
bos_data = pd.concat([X.reset_index(drop=True), y], axis=1)

In [21]:
bos_folds = kfolds(bos_data, 5)

### Perform a linear regression for each fold, and calculate the training and test error

Perform linear regression on each and calculate the training and test error.

In [25]:
from sklearn.metrics import mean_squared_error
test_errs = []
train_errs = []
k=5

for n in range(k):
    # Split in train and test for the fold
    train = pd.concat([fold for i, fold in enumerate(bos_folds) if i!=n])
    test = bos_folds[n]
    # Fit a linear regression model
    linreg.fit(train[X.columns], train[y.columns])
    #Evaluate Train and Test Errors
    y_hat_train = linreg.predict(train[X.columns])
    y_hat_test = linreg.predict(test[X.columns])
    train_mse = mean_squared_error(train[y.columns], y_hat_train)
    test_mse = mean_squared_error(test[y.columns], y_hat_test)
    test_errs.append(test_mse)
    train_errs.append(train_mse)
print(train_errs)
print(test_errs)

[17.918567054246296, 17.3577081046629, 15.545678258525871, 11.037622389644582, 17.23404426556592]
[13.016192102045682, 14.62832183142466, 24.814329971682156, 55.241077726377355, 19.022337999169526]


## Cross-Validation using Scikit-Learn

This was a bit of work! Now, let's perform 5-fold cross-validation to get the mean squared error through scikit-learn. Let's have a look at the five individual MSEs and explain what's going on.

In [29]:
from sklearn.model_selection import cross_val_score
cv_5_results = cross_val_score(linreg, X, y, cv=5, scoring='neg_mean_squared_error')
print(cv_5_results)

[-13.0161921  -14.62832183 -24.81432997 -55.24107773 -19.022338  ]


Next, calculate the mean of the MSE over the 5 cross-validations and compare and contrast with the result from the train-test-split case.

In [30]:
cv_5_results_mean = np.mean(cross_val_score(linreg, X, y, cv=5, scoring='neg_mean_squared_error'))
cv_5_results_mean


-25.344451926139918

The cross-validation indicates our model performs worse than indicated by the train-test-split.

##  Summary 

Congratulations! You now practiced your knowledge on k-fold crossvalidation!